In [ ]:
! source /global/common/software/trn018/init-training-pyhpc-2025.sh

In [ ]:
!unset SLURM_NTASKS_PER_NODE

In [ ]:
%%writefile hello_world.py

from mpi4py import MPI

print("Hello World!")

In [ ]:
!srun -n 2 -c 2 python hello_world.py

In [ ]:
!srun -n 8 -c 8 python hello_world.py